# Modelos e Estratégias de Trading
## Ajustando um modelo VAR à PETR4

Uirá Caiado. 05 de Setembro, 2016

**Resumo**

*Uma das principais necessidades em trading é realizar algum tipo de previsão sobre o estado futuro do instrumento operado para que possamos realizar operações. Como um dos objetivos da análise de séries temporais multivariadas é a realização de forecasts, nesta atividade vou implementar o modelo de Vetor Autoregressivo (VAR) para esta finalidade. Então, vou ajustar o modelo aos retornos e informações de book da PETR4 e analisar a performance realizando um back-test em um parte dos dados coletados aue ainda não tenha sido utilizada.*

## 1. Introdução

Nesta sessão vou ...


### 1.1. Modelagem ...

bla bla

In [1]:
import pandas as pd

In [10]:
df = pd.read_excel('data/VAR3_inputs_e_outputs.xlsx', header=None)

In [11]:
df.head()

,0,1,2,3,4
0,0.004082,0.007782,0.033859,0.023865,0.041760
1,0.012377,0.019575,0.052446,0.056654,-0.010573
2,-0.006921,-0.011853,-0.005707,-0.017716,-0.019980
3,0.000579,0.005753,-0.020576,-0.004274,-0.036814
4,0.001156,0.011407,-0.003350,0.003459,0.000000


In [12]:
df.shape

(200, 5)

## 2. Implementando o Modelo

Nesta sessão detalharemos e implementaremos ...

### 2.1. Bla

bla bla


## 4. Conclusão

bla bla




## 5. Últimas Considerações

bla bla


*Style notebook and change matplotlib defaults*

In [5]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [6]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set2", 10))